In [2]:
from typing import Callable, Any, List
import multiprocessing as mp
from multiprocessing import Queue as MQueue
from queue import Queue
from threading import Thread
import pyring as pr
from pyring import WaitingBlockingRingBuffer, SingleProducerDisruptor
import time

%load_ext autoreload
%autoreload 2

from qubx import logger
from qubx.core.basics import Trade
from qubx.utils import Stopwatch

def run_in_thread(method: Callable, *arguments, **kwargs) -> Thread:
    (t := Thread(target=method, args=arguments, kwargs=kwargs)).start()
    return t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test throughput

In [7]:
sw = Stopwatch()

class DataPumper:
    queue: Queue
    def __init__(self, type='q', ring_size=10000):
        self.type = type
        if type == 'q':
            self.queue = Queue()
        else:
            self.queue = pr.SingleProducerDisruptor(ring_size)

    def pump(self, n):
        sw.start(f'{self.type.upper()}:pump_batch')
        self.queue.put('S')
        for i in range(n):
            self.queue.put(Trade(100, i + 0.1, i * 100))
        self.queue.put('F')
        sw.stop(f'{self.type.upper()}:pump_batch')

class DataReader:
    def __init__(self, pumper: DataPumper):
        self.queue = pumper.queue
        self.type = pumper.type
        self._run = True

    def _read_queue(self):
        print("START reading queue")
        while self._run:
            data = self.queue.get()
            if isinstance(data, str):
                if data == 'S':
                    sw.start('Q:read_batch')
                    print("batch reading started ...")
                elif data == 'F':
                    sw.stop('Q:read_batch')
                    print("batch finished")

    def _read_disr(self):
        print("START reading disruptor")
        sbscr = self.queue.subscribe()
        while self._run:
            idx, data = sbscr.next()
            if isinstance(data, str):
                if data == 'S':
                    sw.start('D:read_batch')
                    print("batch reading started ...")
                elif data == 'F':
                    sw.stop('D:read_batch')
                    print("batch finished")
        sbscr.unregister()

    def read(self):
        if self.type=='q':
            self._read_queue()
        else:
            self._read_disr()

    def run(self):
        return run_in_thread(self.read)

    def stop(self):
        self._run = False
        self.queue.put(None)

In [12]:
# PyRing
dp = DataPumper('d')
rd = DataReader(dp)
rd.run()
dp.pump(1_000_000)
dp.pump(1_000_000)
dp.pump(1_000_000)
rd.stop()
# dp.pump(1)

START reading disruptor
batch reading started ...
batch finished
batch reading started ...
batch finished
batch reading started ...


In [ ]:
# Standard Queue
dp = DataPumper('q')
rd = DataReader(dp)
rd.run()
dp.pump(1_000_000)
dp.pump(1_000_000)
dp.pump(1_000_000)
rd.stop()

In [14]:
logger.info('Tests' + str(sw))

2024-03-18 10:53:58.319 [ ℹ️ ] Tests
	D:pump_batch took 2.0618683 secs
	D:read_batch took 2.0830948 secs
	Q:pump_batch took 4.1221722 secs
	Q:read_batch took 4.3047418 secs


In [7]:
sw.reset()

# Test pyring


In [57]:
import typing
from pyring.ring_buffer import SimpleFactory, RingBufferInternal, RingFactory
from pyring.disruptor import DisruptorMethods
from multiprocessing import Value, Lock as MpLock

In [ ]:
# create ring buffer
ring_buffer = pr.RingBuffer()

# add to ring
ring_buffer.put("Something new!")

# get latest from ring
sequence, value = ring_buffer.get_latest()
print(sequence, value) # 0 Something new!

0 Something new!


In [ ]:
disruptor = pr.SingleProducerDisruptor()

subscriber = disruptor.subscribe()
# subscriber_two = disruptor.subscribe()

for i in range(100):
    disruptor.put(i ** 2)
    sequence_one, res_one = subscriber.next()
    # sequence_two, res_two = subscriber_two.next()
    print(sequence_one, res_one)

# releases the subscribers barriers and allows disruptor to continue
subscriber.unregister()
# subscriber_two.unregister()

In [86]:
def prod1(disruptor):
    # time.sleep(0.1)
    # print('1 START')
    for i in range(10):
        disruptor.put(f" 1 -------------- {i}")
        # time.sleep(0.1)
    # print('1 DONE')

def prod2(disruptor):
    # print('2 START')
    for i in range(10):
        disruptor.put(f" 2 . . . . . . .  {-i}")
        # time.sleep(0.1)
    # print('2 DONE')

In [59]:
from disruptors import DisruptorSubscriber, MultiProducerDisruptor

In [120]:
disruptor = MultiProducerDisruptor(size=4)
subscriber = disruptor.subscribe()

run_in_thread(prod2, disruptor);
run_in_thread(prod1, disruptor);
while True:
    idx, res = subscriber.next(1)
    print(idx, res)

 PUT ~  2 . . . . . . .  0
 PUT ~  2 . . . . . . .  -1
 PUT ~  1 -------------- 0
0  2 . . . . . . .  0
 PUT ~  2 . . . . . . .  -2
 PUT ~  1 -------------- 1
1  2 . . . . . . .  -1
2  1 -------------- 0
 PUT ~  2 . . . . . . .  -3
3  2 . . . . . . .  -2
 PUT ~  1 -------------- 2
4  1 -------------- 1
 PUT ~  1 -------------- 3
 PUT ~  2 . . . . . . .  -4
5  1 -------------- 2
6  2 . . . . . . .  -3
 PUT ~  1 -------------- 4
7  1 -------------- 3
 PUT ~  2 . . . . . . .  -5
 PUT ~  1 -------------- 5
8  2 . . . . . . .  -4
9  1 -------------- 4
 PUT ~  2 . . . . . . .  -6
10  2 . . . . . . .  -5
 PUT ~  1 -------------- 6
 PUT ~  2 . . . . . . .  -7
11  1 -------------- 5
12  2 . . . . . . .  -6
 PUT ~  1 -------------- 7 PUT ~  2 . . . . . . .  -8
13  1 -------------- 6
14  2 . . . . . . .  -7

 PUT ~  2 . . . . . . .  -9
15  2 . . . . . . .  -8
 PUT ~  1 -------------- 8
16  2 . . . . . . .  -9
17  1 -------------- 7
 PUT ~  1 -------------- 918  1 -------------- 8

19  1 ---------

SequenceNotFound: 